In [1]:
import socket
import threading
import os
import json
import numpy as np
import tensorflow as tf
from numpy import float32

from Config import Config

In [2]:
class Server:
    def __init__(self,cfg):
        self.cfg = cfg
        self.s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        self.accept_connections()
    
    def accept_connections(self):
        ip = '' 
        port = self.cfg.server_port

        print('Running on IP: '+ip)
        print('Running on port: '+str(port))

        self.s.bind((ip,port))
        self.s.listen(100)

        while 1:
            c, addr = self.s.accept()
            # print(c)

            threading.Thread(target=self.handle_client,args=(c,addr,)).start()

    def handle_client(self,c,addr):
        received_data = c.recv(1024).decode()
        print("received_data="+received_data)
        obj = json.loads(received_data)
        print(obj)
        tensor_shape = obj['data_shape']
        c.send("OK".encode())

        write_name = 'test' + '.recd'
        if os.path.exists(write_name): os.remove(write_name)

        total_data = 0
        msg = bytearray()
        while 1:
            data = c.recv(1024)
            # print(type(data))
            msg.extend(data)
            if not data:
                break
            total_data += len(data)
        
        print('total size of msg=%d' % (len(msg)))
        
        generated_np_array = np.frombuffer(msg, dtype=float32)

        generated_image_np_array = generated_np_array.reshape(tensor_shape)
        generate_image_tensor = tf.convert_to_tensor(generated_image_np_array, dtype=tf.float32)
        print(generate_image_tensor.shape)



In [3]:
# receive_buffer = np.empty()

In [4]:
cfg = Config()
server = Server(cfg)

In WSL
Running on IP: 
Running on port: 5001
received_data={"data_type": "data", "file_name": "/home/suphale/WorkSpace/000000350497.jpg", "data_size": 1072812, "data_shape": [299, 299, 3]}
{'data_type': 'data', 'file_name': '/home/suphale/WorkSpace/000000350497.jpg', 'data_size': 1072812, 'data_shape': [299, 299, 3]}
total size of msg=1072812
(299, 299, 3)
